### YOLO & Mediapipe & LSTM/GRU를 사용한 모델
* 원본: https://github.com/seonydg/LSTM-for-Anomaly-Detection

In [1]:
%pip install mediapipe==0.10.11
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
import random
import gc
import glob
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from ultralytics import YOLO
from IPython.display import Image, clear_output
from torch.cuda import memory_allocated, empty_cache
from torch.optim import Adam
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

%matplotlib inline

In [3]:
data_root = r'D:\project\New_Data'
file_root = rf'{data_root}\Image'
project_name = 'fall_detection'
print(file_root)
train_root = rf'{data_root}\train'
valid_root = rf'{data_root}\valid'
test_root = rf'{data_root}\test'
cls_list = ['Normal', 'Fall']
cls_filename_list = ['N', 'BY', 'FY', 'SY']

image_root = f'{train_root}\\images' # resize 후(640*640) 이미지 경로
label_root = f'{train_root}\\labels'

image_root_4k = r'D:\project\Data\Data\Data\Training\SourceData\TS\Image' # 원본 이미지(3840*2160) 경로. 자신의 경로에 맞춰서 사용 바람...

image_list = sorted(glob.glob(f'{image_root}\\*.jpg'))
print(len(image_list))
label_list = [x.replace('images', 'labels').replace('JPG', 'jpg').replace('jpg', 'txt') for x in image_list]
print(len(image_list), len(label_list))

D:\project\New_Data\Image
67840
67840 67840


# **1. 사용할 이미지 선택 & mediapipe로 이미지의 pose 데이터 추출**
* bbox의 경우 덕휘님이 주신 yolo.pt 데이터를 사용한 후 1.4배 늘려서 사용

In [4]:
# 예전(json의 bbox를 그대로 사용할 때)에 쓴 코드. ***무시해도 좋음***
def image_sampling_old(image_root, label_root, sample_num):
    image_cls_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    image_names_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    image_sampled_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    label_sampled_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    for cls in cls_filename_list:
        image_cls_dict[cls] = glob.glob(f'{image_root}\\*_{cls}_*')
        image_names_dict[cls] = list(set([x.split('\\')[-1].split('_')[0] for x in image_cls_dict[cls]]))
        div = 2 if cls == 'N' else 6
        image_names_dict[cls] = random.sample(image_names_dict[cls], round((sample_num / 80) / div))
        for i in tqdm(image_names_dict[cls]):
            image_sampled_dict[cls].extend(sorted(glob.glob(f'{image_root}\\{i}_*')))
            label_sampled_dict[cls].extend(sorted(glob.glob(f'{label_root}\\{i}_*')))
    print([image_sampled_dict[cls][:10] for cls in cls_filename_list])
    print([label_sampled_dict[cls][:10] for cls in cls_filename_list])
    print([len(image_sampled_dict[cls]) for cls in cls_filename_list])
    return image_sampled_dict, label_sampled_dict

In [5]:
# ....\train\images\(image_name)_I00X.jpg 형태의 경로에 대하여 사용.
def image_sampling(image_root, sample_num):
    image_cls_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    image_names_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    image_sampled_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    for cls in cls_filename_list:
        image_cls_dict[cls] = glob.glob(f'{image_root}\\*_{cls}_*')
        image_names_dict[cls] = list(set([x.split('\\')[-1].split('_')[0] for x in image_cls_dict[cls]]))
        div = 2 if cls == 'N' else 6
        image_names_dict[cls] = random.sample(image_names_dict[cls], round((sample_num / 80) / div))
        for i in tqdm(image_names_dict[cls]):
            image_sampled_dict[cls].extend(sorted(glob.glob(f'{image_root}\\{i}_*')))
    print([image_sampled_dict[cls][:10] for cls in cls_filename_list])
    print([len(image_sampled_dict[cls]) for cls in cls_filename_list])
    return image_sampled_dict

In [6]:
# ....\N\N\(image_name)\(image_name)_I00X.jpg 형태의 경로에 대하여 사용.
def image_sampling2(image_root, sample_num):
    image_cls_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    image_names_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    image_sampled_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    for cls in cls_filename_list:
        image_cls_dict[cls] = glob.glob(f'{image_root}\\*\\{cls}\\*')
        image_names_dict[cls] = list(set([x.split('\\')[-1].split('_')[0] for x in image_cls_dict[cls]]))
        div = 2 if cls == 'N' else 6
        image_names_dict[cls] = random.sample(image_names_dict[cls], round((sample_num / 80) / div))
        for i in tqdm(image_names_dict[cls]):
            image_sampled_dict[cls].extend(sorted(glob.glob(f'{image_root_4k}\\*\\{cls}\\{i}_*\\*')))
        print([image_sampled_dict[cls][:10] for cls in cls_filename_list])
        print([len(image_sampled_dict[cls]) for cls in cls_filename_list])
    return image_sampled_dict

In [141]:
# 원하는 양의 이미지&라벨을 선택 후 각 클래스(N, BY, SY, FY) 형태로 저장
image_dict, label_dict = image_sampling(image_root, label_root, 30000)
len(label_dict['N'])

100%|██████████| 62/62 [00:20<00:00,  3.03it/s]

[['E:\\project\\New_Data\\train\\images\\00873_O_E_N_C1_I001.jpg', 'E:\\project\\New_Data\\train\\images\\00873_O_E_N_C1_I002.jpg', 'E:\\project\\New_Data\\train\\images\\00873_O_E_N_C1_I003.jpg', 'E:\\project\\New_Data\\train\\images\\00873_O_E_N_C1_I004.jpg', 'E:\\project\\New_Data\\train\\images\\00873_O_E_N_C1_I005.jpg', 'E:\\project\\New_Data\\train\\images\\00873_O_E_N_C1_I006.jpg', 'E:\\project\\New_Data\\train\\images\\00873_O_E_N_C1_I007.jpg', 'E:\\project\\New_Data\\train\\images\\00873_O_E_N_C1_I008.jpg', 'E:\\project\\New_Data\\train\\images\\00873_O_E_N_C1_I009.jpg', 'E:\\project\\New_Data\\train\\images\\00873_O_E_N_C1_I010.jpg'], ['E:\\project\\New_Data\\train\\images\\01346_O_F_BY_C1_I001.jpg', 'E:\\project\\New_Data\\train\\images\\01346_O_F_BY_C1_I002.jpg', 'E:\\project\\New_Data\\train\\images\\01346_O_F_BY_C1_I003.jpg', 'E:\\project\\New_Data\\train\\images\\01346_O_F_BY_C1_I004.jpg', 'E:\\project\\New_Data\\train\\images\\01346_O_F_BY_C1_I005.jpg', 'E:\\project\\Ne

15040

In [25]:
# 원하는 양의 이미지&라벨을 선택 후 각 클래스(N, BY, SY, FY) 형태로 저장
image_dict = image_sampling2(image_root_4k, 12000)
len(image_dict['N'])

100%|██████████| 75/75 [00:05<00:00, 13.01it/s]


[['D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I001.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I002.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I003.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I004.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I005.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I006.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I007.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I008.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O

100%|██████████| 25/25 [00:02<00:00, 11.84it/s]


[['D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I001.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I002.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I003.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I004.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I005.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I006.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I007.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I008.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O

100%|██████████| 25/25 [00:02<00:00, 11.22it/s]


[['D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I001.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I002.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I003.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I004.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I005.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I006.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I007.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I008.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O

100%|██████████| 25/25 [00:02<00:00, 10.88it/s]

[['D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I001.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I002.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I003.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I004.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I005.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I006.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I007.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O_E_N_C1_I008.jpg', 'D:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\00774_O_E_N_C1\\00774_O

6000

In [14]:
# 상수값 설정
BATCH_SIZE = 8
EPOCH = 10
NUM_LAYERS = 1
n_CONFIDENCE = 0.3
# resolution: 
#   (640, 640): resize한 이미지 사용시
#   (3840, 2160): 원본 이미지 사용시
resolution = (3840, 2160)
my_moniter_resolution = (1920, 1080)

mp_pose = mp.solutions.pose
# pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
yolo_model = YOLO(r'D:\project\code\Project_humanFall\runs\detect\human_fall_s30\weights\best.pt', verbose=False)
mp_drawing = mp.solutions.drawing_utils
attention_dot = [n for n in range(33)]


draw_line_list = [[11, 13], [13, 15], [15, 21], [15, 19], [15, 17], [17, 19], [12, 14], [14, 16],
             [16, 22], [16, 20], [16, 18], [18, 20], [23, 25], [25, 27], [24, 26], [26, 28],
             [11, 12], [11, 23], [23, 24], [12, 24], [9, 10], [0, 5], [0, 2], [5, 8], [2, 7],
             [28, 30], [30, 32], [28, 32], [27, 29], [29, 31], [27, 31]]
COLOR_PINK = (255, 51, 153)
COLOR_WHITE = (255, 255, 255)

In [42]:
# 아래의 get_pose_with_yolo에서 사용
def make_imshow_samples(
        draw_line_list,
        draw_line_dic,
        img_ori,
        img_pose,
        bx1, by1, bx2, by2,
        img_src,
        isFailed,
        resolution
        ):
    img_ori = cv2.resize(img_ori, tuple(np.array(my_moniter_resolution) // 3) if (resolution[0] > 1600 or resolution[1] > 900) else resolution)
    cv2.putText(img_ori, img_src.split('\\')[-1], (5, 30), cv2.FONT_HERSHEY_DUPLEX, 1, COLOR_PINK, lineType=cv2.LINE_AA)
    if isFailed == 'Detected':
        for line in draw_line_list:
            lx1, ly1 = draw_line_dic[line[0]][0], draw_line_dic[line[0]][1]
            lx2, ly2 = draw_line_dic[line[1]][0], draw_line_dic[line[1]][1]
            img_pose = cv2.line(img_pose, (bx1 + lx1, by1 + ly1), (bx1 + lx2, by1 + ly2), (0, 255, 0), 15)
            
        img_pose = cv2.rectangle(img_pose, (bx1, by1), (bx2, by2), (0, 255, 0), 15)
        img_pose = cv2.resize(img_pose, tuple(np.array(my_moniter_resolution) // 3) if (resolution[0] > 1600 or resolution[1] > 900) else resolution)
        return [img_ori, img_pose, isFailed, 0]
    elif isFailed == 'Failed!: Pose Out of BBOX':
        for line in draw_line_list:
            lx1, ly1 = draw_line_dic[line[0]][0], draw_line_dic[line[0]][1]
            lx2, ly2 = draw_line_dic[line[1]][0], draw_line_dic[line[1]][1]
            img_pose = cv2.line(img_pose, (bx1 + lx1, by1 + ly1), (bx1 + lx2, by1 + ly2), (0, 255, 0), 15)
            
        img_pose = cv2.rectangle(img_pose, (bx1, by1), (bx2, by2), (0, 255, 0), 15)
        img_pose = cv2.resize(img_pose, tuple(np.array(my_moniter_resolution) // 3) if (resolution[0] > 1600 or resolution[1] > 900) else resolution)
        return [img_ori, False, isFailed, img_pose]
    else:
        return [img_ori, False, isFailed, 0]

In [43]:
# 제가 예전에 쓴 방법(bbox 데이터를 json 파일에서 바로 받아 쓰기). ***무시해도 좋음***
def get_pose(image_list: list, label_list: list, attention_dot: list, draw_line_list: list, isSample=False, sample_size=0):
    if isSample:
        print('sample mod')
        print(len(image_list))
        plt.figure(figsize=(8, 4*sample_size))
    frame_length = 10 # LSTM 모델에 넣을 frame 수
    xy_list_list = []
    omission_cnt = 0
    for i, (img_src, lbl) in enumerate((zip(image_list, label_list))):
        if label_type == 'yolo':
            with open(lbl, 'r', encoding='utf-8') as f:
                label_str = f.read()
                bbox = list(map(float, label_str.split(' ')[1:]))

            bbox = [int(x * 640) for x in bbox]
            bbox = [
                max(int(bbox[0] - bbox[2] / 2 - 30), 0),
                max(int(bbox[1] - bbox[3] / 2 - 30), 0),
                min(int(bbox[0] + bbox[2] / 2 + 30), 640),
                min(int(bbox[1] + bbox[3] / 2 + 30), 640),
            ]
            img = cv2.imread(img_src)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            if isSample:
                bbox_img = cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 0), 2)
                cv2.putText(bbox_img, img_src.split('\\')[-1], (5, 30), cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), lineType=cv2.LINE_AA)
                plt.subplot(sample_size, 2, (2*i+1))
                plt.imshow(bbox_img)

        
        """ Yolov5 바운딩 박스 좌표 안에서 mediapipe Pose 추출"""

        c_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]] # 바운딩 박스 좌표/
        del img
        results = pose.process(c_img) # Yolov5 바운딩 박스 좌표 안에서 'mp_pose' 좌표

        if not results.pose_landmarks: continue
        idx = 0
        draw_line_dic = {}
        xy_list = []
        # 33 반복문 진행 : 33개 중 18개의 dot
        for x_and_y in results.pose_landmarks.landmark:
            if idx in attention_dot:
                xy_list.append(x_and_y.x)
                xy_list.append(x_and_y.y)

                x, y = int(x_and_y.x * (bbox[2] - bbox[0])), int(x_and_y.y * (bbox[3] - bbox[1]))
                draw_line_dic[idx] = [x, y]
            idx += 1

        if len(xy_list) != len(attention_dot) * 2:
            print('Error : attention_dot 데이터 오류')

        xy_list_list.append(xy_list)

        """mediapipe line 그리기 부분 : 데이터 추출(dot) 확인용"""
        if isSample:
            for line in draw_line_list:
                x1, y1 = draw_line_dic[line[0]][0], draw_line_dic[line[0]][1]
                x2, y2 = draw_line_dic[line[1]][0], draw_line_dic[line[1]][1]
                c_img = cv2.line(c_img, (x1, y1), (x2, y2), (0, 255, 0), 4)
            plt.subplot(sample_size, 2, (2*i+2))
            plt.imshow(c_img)
        del c_img
        
    # 부족한 프레임 수 맞추기
    if isSample:
        if len(xy_list_list) < len(image_list) // 2:
            return False
        elif len(xy_list_list) < len(image_list):
            f_ln = len(image_list) - len(xy_list)
            for _ in range(f_ln):
                xy_list_list.append(xy_list_list[-1])
    else:
        if len(xy_list_list) < 5:
            return False, 5 - len(xy_list_list)
        elif len(xy_list_list) < 10:
            f_ln = frame_length - len(xy_list)
            for _ in range(f_ln):
                xy_list_list.append(xy_list_list[-1])
                omission_cnt = 1
    
    if isSample:
        plt.show()
    return xy_list_list, omission_cnt

In [44]:
# 덕휘님의 yolo .pt 파일을 사용하여 10장의 시간적으로 연결된 이미지에서 bbox를 찾고,
# 그 bbox 안에서 mediapipe를 통해 pose 데이터를 추출하고,
# 10장의 pose데이터를 list로 저장 + 인식에 실패한 데이터에 관한 정보를 출력
def get_pose_with_yolo(image_list: list, attention_dot: list, draw_line_list: list, resolution: tuple, isSample=False, sample_size=0):
    with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5) as pose:
        # if isSample: ==> get_pose_sample_4k 함수를 통해서 호출되었을 시 동작하는 부분 >> cv2.imshow()를 위한 동작 수행
        if isSample: 
            print('sample mod')
            print(len(image_list))
            imshow_img_list = []
        pose_coor_1case = [] # 한 건(연결된 이미지 10장) 안에서의 pose_coor을 모은 것
        omission_cnt = 0
        for i, img_src in enumerate(image_list):
            img = cv2.imread(img_src)
            if isSample: img_ori = img.copy()
            results = yolo_model(img, verbose=False)
            for r in results:
                boxes = r.boxes
                for box in boxes:
                    # 사람 클래스인 경우에만 처리 (YOLO 클래스 인덱스에 따라 조정 필요)
                    # if int(box.cls) == 0:  # 0은 보통 'person' 클래스
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    bx1, by1, bx2, by2 = (
                        max(int(x1 - (x2 - x1)*0.2), 0),
                        max(int(y1 - (y2 - y1)*0.2), 0),
                        min(int(x2 + (x2 - x1)*0.2), 3840),
                        min(int(y2 + (y2 - y1)*0.2), 2160),
                    )
                    # 바운딩 박스 추출
                    person_image = img[by1:by2, bx1:bx2]
                    
                    # MediaPipe로 포즈 추정
                    results_pose = pose.process(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))

                    idx = 0 
                    draw_line_dic = {} # 샘플 보여주기용 dict
                    pose_coor = [] # 한 이미지에서 관절의 xy좌표값의 list ex) [[12, 44], [122, 334], ... , [394, 664]]

                    # pose 인식 여부 확인 
                    if results_pose.pose_landmarks:
                        landmarks = results_pose.pose_landmarks

                        # pose 좌표값 list 생성
                        for pose_xy in landmarks.landmark:
                            if idx in attention_dot:
                                pose_coor.append(pose_xy.x)
                                pose_coor.append(pose_xy.y)
                                x, y = int(pose_xy.x * (bx2 - bx1)), int(pose_xy.y * (by2 - by1))
                                draw_line_dic[idx] = [x, y]
                            idx += 1

                        if len(pose_coor) != len(attention_dot) * 2:
                            print('Error : attention_dot 데이터 오류')

                        # pose 좌표값중 어느 하나라도 바운딩박스를 초과할 시, 인식 실패로 판정
                        if all(list(map(lambda x: 0 < x < 1, pose_coor))) == False:
                            pose_coor_1case.append(False)
                            if isSample:
                                img_pose = img.copy()
                                isFailed = 'Failed!: Pose Out of BBOX'
                                omission_cnt += 1
                                imshow_img_list.append(make_imshow_samples(draw_line_list, draw_line_dic, img_ori, img_pose, bx1, by1, bx2, by2, img_src, isFailed, resolution))
                                break
                        
                        # 결과값 list에 추가
                        pose_coor_1case.append(pose_coor)

                        if isSample:
                            img_pose = img.copy()
                            isFailed = 'Detected'
                            imshow_img_list.append(make_imshow_samples(draw_line_list, draw_line_dic, img_ori, img_pose, bx1, by1, bx2, by2, img_src, isFailed, resolution))

                    # 인식 실패 시 False로 표시
                    else:
                        pose_coor_1case.append(False)
                        omission_cnt += 1
                        img_pose = img.copy()
                        if isSample:
                            isFailed = 'Failed!: Can\'t detect by mediapipe'
                            imshow_img_list.append(make_imshow_samples(draw_line_list, draw_line_dic, img_ori, img_pose, bx1, by1, bx2, by2, img_src, isFailed, resolution))
                            
                    break # 우리가 사용하는 .pt 파일은 0: Normal, 1: Fall인데 바운딩박스만 잡으면 큰 상관 없으므로 둘 중 하나만 사용
    
    # mediapipe로 인식 실패한 이미지의 pose데이터 채우기
    if pose_coor_1case.count(False) >= 5: # 10개 중 5개 이상이 인식 실패했을 시 해당 10장 전체를 실패 처리
        if isSample:
            print('more than 5 images fail to be detected!')
        return False, 1, 1, pose_coor_1case.count(False)
    max_len = (len(pose_coor_1case) - 1)

    while pose_coor_1case.count(False) > 0:
        om_idx = pose_coor_1case.index(False) # 인식 실패한 이미지의 위치를 저장

        if om_idx == 0: # 맨 앞의 이미지가 인식 실패했을 시 그 뒤의 성공한 이미지의 데이터로 채움
            while pose_coor_1case[om_idx + 1] == False:
                om_idx += 1
            pose_coor_1case[om_idx] = pose_coor_1case[om_idx + 1].copy()
            if isSample:
                imshow_img_list[om_idx][1] = imshow_img_list[om_idx + 1][1].copy()

        else: # 그 외에는 그 앞의 성공한 이미지의 데이터로 채움
            pose_coor_1case[om_idx] = pose_coor_1case[om_idx - 1].copy()
            if isSample:
                imshow_img_list[om_idx][1] = imshow_img_list[om_idx - 1][1].copy()

    # sample 이미지를 imshow
    if isSample:
        imshow_img_iter = iter(imshow_img_list)
        while True:
            try:
                imshow_img = imshow_img_iter.__next__()
                cv2.putText(imshow_img[1], imshow_img[2], (5, 30), cv2.FONT_HERSHEY_DUPLEX, 1, COLOR_PINK, lineType=cv2.LINE_AA)
                cv2.imshow('ori_img', imshow_img[0])
                cv2.imshow('img_with_mediapipe_pose', imshow_img[1])
                if imshow_img[2] ==  'Failed!: Pose Out of BBOX':
                    cv2.imshow('Pose Out of BBOX:', imshow_img[3])
                if cv2.waitKey() & 0xFF == ord('q'):
                    print("사용자에 의해 종료됨")
                    return 'q'
            except StopIteration:
                break
    cv2.destroyAllWindows()
    return pose_coor_1case, min(omission_cnt, 1), 0, omission_cnt

In [45]:
# 이미지 데이터의 일부를 샘플로 잡고, mediapipe로 추출한 pose 데이터를 cv2.imshow로 plot
def get_pose_sample(image_dict: dict, attention_dot: list, draw_line_list: list, resolution: tuple, sample_size: int, sample_start=0):
    cv2.destroyAllWindows()
    image_pose_sample = []
    for cls in cls_filename_list:
        div = 4
        image_pose_sample.extend(image_dict[cls][sample_start:(sample_start+sample_size//div)])
    print(len(image_pose_sample))
    for i in range(sample_size//10):
        q = get_pose_with_yolo(image_pose_sample[i*10:(i+1)*10], attention_dot, draw_line_list, resolution, True, sample_size)
        if q == 'q':
            cv2.destroyAllWindows()
            return

In [47]:
# mediapipe로 감지한 결과 샘플을 보여줌. 아무 키나 눌러서 다음 이미지 확인
# sample_size: 샘플로 보여줄 이미지 수(10의 배수 권장)
# sample_start: 샘플을 시작할 위치 ex) sample_size=40일 때 sample_start=0으로 작동시킨 후 sample_start=40으로 작동시키면 바로 다음 이미지부터 확인 가능
get_pose_sample(image_dict, attention_dot, draw_line_list, resolution, 40, 0)


40
sample mod
10
sample mod
10
sample mod
10
sample mod
10


In [77]:
# [[C1_I0001, C1_I0002, ...C1_I0010], [C2_I0001, ...], ...] 형태로 묶음
def collect_by_case(image_dict, label_dict=0):
    raw_data = []
    image_pose_input = []
    label_pose_input = []
    for cls in cls_filename_list:
        img_name = {'index': 'None', 'direction': 'None'}
        img_bundle = []
        label_bundle = []
        isNewBundle = 0
        dict_iter = zip(tqdm(image_dict[cls]), label_dict[cls]) if label_dict != 0 else tqdm(image_dict[cls])
        for elements in dict_iter:
            if label_dict != 0: 
                img = elements[0]
                lbl = elements[1]
            else: img = elements

            # 처음 값 입력
            if img_name['index'] != img.split('\\')[-1].split('_')[0]:
                img_name['index'] = img.split('\\')[-1].split('_')[0]
                isNewBundle += 1
            if img_name['direction'] != img.split('\\')[-1].split('_')[-2]:
                img_name['direction'] = img.split('\\')[-1].split('_')[-2]
                isNewBundle += 1
            if isNewBundle > 0:
                if img_bundle != []:
                    image_pose_input.append(img_bundle)
                    if label_dict != 0: label_pose_input.append(label_bundle)
                img_bundle = []
                label_bundle = []
                isNewBundle = 0
            img_bundle.append(img)
            if label_dict != 0: label_bundle.append(lbl)


    print(len(image_dict['N'])+len(image_dict['BY'])+len(image_dict['FY'])+len(image_dict['SY']))
    print(image_pose_input[:3])
    print(len(image_pose_input))
    print(len(label_pose_input))

    return image_pose_input, label_pose_input

In [78]:
# mediapipe로 pose data를 list로 추출 후, raw_data에 [{낙상, pose data}, {비낙상, pose data}, ...] 형태로 저장 (4k 원본 이미지, yolo 사용)


omission_included_cnt = 0 # 10장 중 누락된 이미지가 있는 그룹의 개수
totally_omitted_cnt = 0 # 10장 중 5장 이상이 누락되어 사용하지 않는 그룹의 개수
omission_total = 0 # 인식되지 않은 이미지의 개수
image_pose_input, _ = collect_by_case(image_dict) # label(.txt) 파일을 사용할 시 매개변수에 label_dict 추가(label의 bbox 데이터는 사용하지 않기로 했으므로 ***무시해도 됨***)
raw_data = []

for img_list in tqdm(image_pose_input):
    cls = 0 if img_list[0].split('\\')[-1].split('_')[-3] == 'N' else 1
    pose_list, isIncludingOmission, isTotallyOmitted, omission_cnt = get_pose_with_yolo(img_list, attention_dot, draw_line_list, resolution, False)
    if pose_list != False:
        raw_data.append({'key': cls, 'value': pose_list})
    omission_included_cnt += isIncludingOmission
    totally_omitted_cnt += isTotallyOmitted
    omission_total += omission_cnt

print('omission_included_cnt: ', omission_included_cnt)
print('totally_omitted_cnt: ', totally_omitted_cnt)
print('ommision_cnt: ', omission_total)

100%|██████████| 2480/2480 [00:00<00:00, 620044.94it/s]


14960
[['E:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\02607_H_A_N_C1\\02607_H_A_N_C1_I001.jpg', 'E:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\02607_H_A_N_C1\\02607_H_A_N_C1_I002.jpg', 'E:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\02607_H_A_N_C1\\02607_H_A_N_C1_I003.jpg', 'E:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\02607_H_A_N_C1\\02607_H_A_N_C1_I004.jpg', 'E:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\02607_H_A_N_C1\\02607_H_A_N_C1_I005.jpg', 'E:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\02607_H_A_N_C1\\02607_H_A_N_C1_I006.jpg', 'E:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\02607_H_A_N_C1\\02607_H_A_N_C1_I007.jpg', 'E:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\02607_H_A_N_C1\\02607_H_A_N_C1_I008.jpg', 'E:\\project\\Data\\Data\\Data\\Training\\SourceData\\TS\\Image\\N\\N\\02607_H_A_N_C1\\0

100%|██████████| 1492/1492 [24:11<00:00,  1.03it/s]

omission_included_cnt:  500
totally_omitted_cnt:  228
ommision_cnt:  1738


예시 결과
* omission_included_cnt:  500
* totally_omitted_cnt:  228
* ommision_cnt:  1738

# **2. LSTM / GRU 모델 제작**
* 여기서부터는 거의 복붙한 그대로입니다...

In [82]:
print(raw_data[:10])
nd = 0
ad = 0
for i in range(len(raw_data)):
    if raw_data[i]['key'] == 0:
        nd += 1
    else:
        ad += 1
print('normal data:', nd, '| fall data:', ad)

[{'key': 0, 'value': [[0.3894423246383667, 0.2774071991443634, 0.4179619550704956, 0.25207382440567017, 0.436786413192749, 0.2525045573711395, 0.45934757590293884, 0.25359806418418884, 0.3844284415245056, 0.25109001994132996, 0.3767092823982239, 0.24987247586250305, 0.3693655729293823, 0.24794870615005493, 0.5198522806167603, 0.25999006628990173, 0.4026291072368622, 0.2634381651878357, 0.4517938494682312, 0.30087149143218994, 0.3878400921821594, 0.2989742159843445, 0.7034311294555664, 0.34832850098609924, 0.376140832901001, 0.3377785384654999, 0.7682161927223206, 0.5129683613777161, 0.2697863280773163, 0.4523478150367737, 0.5428332090377808, 0.5896903276443481, 0.21723264455795288, 0.3856680393218994, 0.5138347744941711, 0.6226953268051147, 0.19830085337162018, 0.3737668991088867, 0.47496721148490906, 0.6119743585586548, 0.21074557304382324, 0.35672682523727417, 0.4817488193511963, 0.6012148261070251, 0.22543470561504364, 0.35976654291152954, 0.6394978761672974, 0.5286000967025757, 0.4

In [80]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [83]:
class MyDataset(Dataset):
    def __init__(self, seq_list):
        self.X = []
        self.y = []
        for dic in seq_list :
            self.y.append(dic['key'])
            self.X.append(dic['value'])

    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))

    def __len__(self):
        return len(self.X)

In [84]:
split_ratio = [0.8, 0.1, 0.1]
train_len = int(len(raw_data) * split_ratio[0])
val_len = int(len(raw_data) * split_ratio[1])
test_len = len(raw_data) - train_len - val_len

print('{}, {}, {}'.format(train_len, val_len, test_len))

1011, 126, 127


In [85]:
train_dataset = MyDataset(raw_data)
train_data, valid_data, test_data = random_split(train_dataset, [train_len, val_len, test_len])

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE)
val_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [91]:
# LSTM
class skeleton_LSTM(nn.Module):
    def __init__(self):
        super(skeleton_LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=len(attention_dot) * 2, hidden_size=128, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=128, hidden_size=256, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm3 = nn.LSTM(input_size=256, hidden_size=512, num_layers=NUM_LAYERS, batch_first=True)
        self.dropout1 = nn.Dropout(0.1)
        self.lstm4 = nn.LSTM(input_size=512, hidden_size=256, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm5 = nn.LSTM(input_size=256, hidden_size=128, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm6 = nn.LSTM(input_size=128, hidden_size=64, num_layers=NUM_LAYERS, batch_first=True)
        self.dropout2 = nn.Dropout(0.1)
        self.lstm7 = nn.LSTM(input_size=64, hidden_size=32, num_layers=NUM_LAYERS, batch_first=True)
        self.fc = nn.Linear(32,2)

    def forward(self, x) :
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.dropout1(x)
        x, _ = self.lstm4(x)
        x, _ = self.lstm5(x)
        x, _ = self.lstm6(x)
        x = self.dropout2(x)
        x, _ = self.lstm7(x)
        x = self.fc(x[:,-1,:]) # x[배치 크기, 시퀀스 길이, 은닉 상태 크기], [:, -1, :] -> 마지막 시간 단계만 선택

        return x
     

In [86]:
# GRU
# 위의 LSTM class를 그대로 GRU로만 바꿔서 만듦
# 아래의 코드에서 skeleton_LSTM()을 skeleton_GRU()로 바꾸면 사용 가능
class skeleton_GRU(nn.Module):
    def __init__(self):
        super(skeleton_GRU, self).__init__()
        self.gru1 = nn.GRU(input_size=len(attention_dot) * 2, hidden_size=128, num_layers=NUM_LAYERS, batch_first=True)
        self.gru2 = nn.GRU(input_size=128, hidden_size=256, num_layers=NUM_LAYERS, batch_first=True)
        self.gru3 = nn.GRU(input_size=256, hidden_size=512, num_layers=NUM_LAYERS, batch_first=True)
        self.dropout1 = nn.Dropout(0.1)
        self.gru4 = nn.GRU(input_size=512, hidden_size=256, num_layers=NUM_LAYERS, batch_first=True)
        self.gru5 = nn.GRU(input_size=256, hidden_size=128, num_layers=NUM_LAYERS, batch_first=True)
        self.gru6 = nn.GRU(input_size=128, hidden_size=64, num_layers=NUM_LAYERS, batch_first=True)
        self.dropout2 = nn.Dropout(0.1)
        self.gru7 = nn.GRU(input_size=64, hidden_size=32, num_layers=NUM_LAYERS, batch_first=True)
        self.fc = nn.Linear(32,2)

    def forward(self, x) :
        x, _ = self.GRU1(x)
        x, _ = self.GRU2(x)
        x, _ = self.GRU3(x)
        x = self.dropout1(x)
        x, _ = self.GRU4(x)
        x, _ = self.GRU5(x)
        x, _ = self.GRU6(x)
        x = self.dropout2(x)
        x, _ = self.GRU7(x)
        x = self.fc(x[:,-1,:]) # x[배치 크기, 시퀀스 길이, 은닉 상태 크기], [:, -1, :] -> 마지막 시간 단계만 선택

        return x

In [87]:
# 모델 초기화

def init_model():
    global net, loss_fn, optim
    plt.rc('font', size = 10)
    net = skeleton_LSTM().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optim = Adam(net.parameters(), lr=0.0001)

# epoch 카운터 초기화
def init_epoch():
    global epoch_cnt
    epoch_cnt = 0

# 모든 Log를 초기화
def init_log():
    global log_stack, iter_log, tloss_log, tacc_log, vloss_log, vacc_log, time_log
    plt.rc('font', size = 10)
    iter_log, tloss_log, tacc_log, vloss_log, vacc_log = [], [], [], [], []
    time_log, log_stack = [], []

In [88]:
def record_train_log(_tloss, _tacc, _time):
    # Train Log 기록
    time_log.append(_time)
    tloss_log.append(_tloss)
    tacc_log.append(_tacc)
    iter_log.append(epoch_cnt)

def record_valid_log(_vloss, _vacc):
    # Validation Log 기록
    vloss_log.append(_vloss)
    vacc_log.append(_vacc)

def last(log_list):
    # last 안의 마지막 숫자를 반환(print_log 함수에서 사용)
    if len(log_list) > 0:
        return log_list[len(log_list) - 1]
    else:
        return -1

def print_log():
    # 학습 추이 출력 : 소숫점 3자리까지
    train_loss = round(float(last(tloss_log)), 3)
    train_acc = round(float(last(tacc_log)), 3)
    val_loss = round(float(last(vloss_log)), 3)
    val_acc = round(float(last(vacc_log)), 3)
    time_spent = round(float(last(time_log)), 3)

    log_str = 'Epoch: {:3} | T_Loss {:5} | T_Acc {:5} | V_Loss {:5} | V_Acc {:5} | {:5}'.format(last(iter_log), train_loss, train_acc, val_loss, val_acc, time_spent)

    log_stack.append(log_str)
    
    # 학습 추이 그래프 출력
    hist_fig, loss_axis = plt.subplots(figsize=(10, 3), dpi=99)
    hist_fig.patch.set_facecolor('white')

    # Loss Line 구성
    loss_t_line = plt.plot(iter_log, tloss_log, label='Train_Loss', color='red', marker='o')
    loss_v_line = plt.plot(iter_log, vloss_log, label='Valid_Loss', color='blue', marker='s')
    loss_axis.set_xlabel('epoch')
    loss_axis.set_ylabel('loss')

    # Acc, Line 구성
    acc_axis = loss_axis.twinx()
    acc_t_line = acc_axis.plot(iter_log, tacc_log, label='Train_Acc', color='red', marker='+')
    acc_v_line = acc_axis.plot(iter_log, vacc_log, label='Valid_Acc', color='blue', marker='x')
    acc_axis.set_ylabel('accuracy')

    # 그래프 출력
    hist_lines = loss_t_line + loss_v_line + acc_t_line + acc_v_line
    loss_axis.legend(hist_lines, [l.get_label() for l in hist_lines])
    loss_axis.grid()
    plt.title('Learning history until epoch {}'.format(last(iter_log)))
    plt.draw()

    # 텍스트 로그 출력
    clear_output(wait=True)
    plt.show()
    for idx in reversed(range(len(log_stack))):
        print(log_stack[idx])

In [89]:
def clear_memory():
    if device != 'cpu':
        empty_cache()
    gc.collect()

# 학습 알고리즘
def epoch(data_loader, mode = 'train'):
    global epoch_cnt

    # 사용되는 변수 초기화
    iter_loss, iter_acc, last_grad_performed = [], [], False

    # 1 iteration 학습 알고리즘(for문을 나오면 1 epoch 완료)
    for _data, _label in data_loader:
        data, label = _data.to(device), _label.type(torch.LongTensor).to(device)

        # 1. Feed-forward
        if mode == 'train':
            net.train()
        else:
            # 학습때만 쓰이는 Dropout, Batch Mormalization을 미사용
            net.eval()

        result = net(data) # 1 Batch에 대한 결과가 모든 Class에 대한 확률값으로
        _, out = torch.max(result, 1) # result에서 최대 확률값을 기준으로 예측 class 도출( _ : 값 부분은 필요 없음, out : index 중 가장 큰 하나의 데이터)

        # 2. Loss 계산
        loss = loss_fn(result, label) # GT 와 Label 비교하여 Loss 산정
        iter_loss.append(loss.item()) # 학습 추이를 위하여 Loss를 기록

        # 3. 역전파 학습 후 Gradient Descent
        if mode == 'train':
            optim.zero_grad() # 미분을 통해 얻은 기울기를 초기화 for 다음 epoch
            loss.backward() # 역전파 학습
            optim.step() # Gradient Descent 수행
            last_grad_performed = True # for문을 나가면 epoch 카운터 += 1

        # 4. 정확도 계산
        acc_partial = (out == label).float().sum() # GT == Label 인 개수
        acc_partial = acc_partial / len(label) # ( TP / (TP + TM)) 해서 정확도 산출
        iter_acc.append(acc_partial.item()) # 학습 추이를 위하여 Acc. 기록

    # 역전파 학습 후 Epoch 카운터 += 1
    if last_grad_performed:
        epoch_cnt += 1

    clear_memory()

    # loss와 acc의 평균값 for 학습추이 그래프, 모든 GT와 Label 값 for 컨퓨전 매트릭스
    return np.average(iter_loss), np.average(iter_acc)

def epoch_not_finished():
    # 에폭이 끝남을 알림
    return epoch_cnt < maximum_epoch

# **3. 학습 및 테스트**
* 여기서 계속 에러나는 중...
* RuntimeError: stack expects each tensor to be equal size, but got [13, 66] at entry 0 and [10, 66] at entry 1

In [92]:
# Training initialization
init_model()
init_epoch()
init_log()
maximum_epoch = EPOCH

# Training iteration

while epoch_not_finished():
    start_time = time.time()

    tloss, tacc = epoch(train_loader, mode = 'train')

    end_time = time.time()
    time_taken = end_time - start_time
    record_train_log(tloss, tacc, time_taken)

    with torch.no_grad():
        vloss, vacc = epoch(val_loader, mode = 'val')
        record_valid_log(vloss, vacc)

    print_log()

print('\n Training completed!')

RuntimeError: stack expects each tensor to be equal size, but got [13, 66] at entry 0 and [10, 66] at entry 1

In [ ]:
# 정확도 검증
with torch.no_grad():
    test_loss, test_acc = epoch(test_loader, mode = 'test')
    test_acc = round(test_acc, 4)
    test_loss = round(test_loss, 4)
    print('Test Acc.: {}'.format(test_acc))
    print('Test Loss: {}'.format(test_loss))

In [ ]:
# 테스트 이미지 선택
# 작성중...

In [ ]:
# 실제 이미지로 테스트